In [1]:
import os
import rasterio
from rasterio.windows import Window
import numpy as np

In [2]:
img_path = './Datos/WCS_land_use/Imagery/wcs_orinoquia_sr_2013_2014.tif'
out_dir = './Datos/WCS_land_use/train_200218'
tile_size = 500

os.makedirs(out_dir, exist_ok=True)

with rasterio.open(img_path) as src:
    width = src.width
    height = src.height
    for i in range(0, width, tile_size):
        for j in range(0, height, tile_size):
            w = min(tile_size, width - i)
            h = min(tile_size, height - j)
            window = Window(i, j, w, h)
            transform = src.window_transform(window)
            tile = src.read(window=window)
            tile_name = f"tile_{i}_{j}.tif"
            out_path = os.path.join(out_dir, tile_name)
            profile = src.profile
            profile.update({
                'height': h,
                'width': w,
                'transform': transform
            })
            with rasterio.open(out_path, 'w', **profile) as dst:
                dst.write(tile)

In [3]:
import geopandas as gpd
from rasterio.features import rasterize

label_path = './Datos/villavicencioWCS.gpkg'  # Usa tu ruta relativa
gdf = gpd.read_file(label_path)

mask_dir = './Datos/masks'
os.makedirs(mask_dir, exist_ok=True)

with rasterio.open(img_path) as src:
    width = src.width
    height = src.height
    for i in range(0, width, tile_size):
        for j in range(0, height, tile_size):
            w = min(tile_size, width - i)
            h = min(tile_size, height - j)
            window = Window(i, j, w, h)
            transform = src.window_transform(window)
            bounds = rasterio.windows.bounds(window, src.transform)
            # Recorta los polígonos al tile
            tile_gdf = gdf.cx[bounds[0]:bounds[2], bounds[1]:bounds[3]]
            if tile_gdf.empty:
                mask = np.zeros((h, w), dtype=np.uint8)
            else:
                # Cambia 'tu_campo_clase' por el nombre real de tu columna de clase
                mask = rasterize(
                    [(geom, value) for geom, value in zip(tile_gdf.geometry, tile_gdf['Class_WCS_coarse'])],
                    out_shape=(h, w),
                    transform=transform,
                    fill=0,
                    dtype=np.uint8
                )
            mask_name = f"mask_{i}_{j}.tif"
            out_path = os.path.join(mask_dir, mask_name)
            profile = src.profile
            profile.update({
                'height': h,
                'width': w,
                'transform': transform,
                'count': 1,
                'dtype': np.uint8
            })
            with rasterio.open(out_path, 'w', **profile) as dst:
                dst.write(mask, 1)